In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from sqlalchemy import create_engine

In [ ]:
server = r"DESKTOP-5VCHN8F\NODESERVER"
database = "Sales Analysis"
driver = "ODBC Driver 17 for SQL Server"

engine = create_engine(
    f"mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes"
)

In [ ]:
os.makedirs("visuals", exist_ok=True)

In [ ]:
# ----- Sales by Category -----
query_category = """
select Category,
SUM(List_price * Quantity * (1-(Discount_Percent/100))) as TotalSales
from Order_Sales
group by Category
order by TotalSales desc;
"""
df_category = pd.read_sql(query_category, engine)
plt.bar(df_category["Category"], df_category["TotalSales"])
plt.title("Sales by Category")
plt.savefig("visuals/sales_by_category.png")
plt.close()

In [ ]:
query_top = """
select top 10 Product_Id,
SUM(List_price * Quantity * (1-(Discount_Percent/100))) as TotalSales
from Order_Sales
group by Product_Id
order by TotalSales desc;
"""
df_top = pd.read_sql(query_top, engine)
plt.bar(df_top["Product_Id"].astype(str), df_top["TotalSales"])
plt.title("Top 10 Products by Revenue")
plt.xticks(rotation=45)
plt.savefig("visuals/top10_products.png")
plt.close()

In [ ]:
# ----- Sales by Region -----
query_region = """
select Region,
SUM(List_price * Quantity * (1-(Discount_Percent/100))) as TotalSales
from Order_Sales
group by Region
order by TotalSales desc;
"""
df_region = pd.read_sql(query_region, engine)
plt.bar(df_region["Region"], df_region["TotalSales"])
plt.title("Sales by Region")
plt.savefig("visuals/sales_by_region.png")
plt.close()

In [ ]:
# ----- Monthly Sales Trend -----
query_monthly = """
select Year(Order_Date) as Year,
Month(Order_Date) as Month,
datename(month,Order_Date) as MonthName,
SUM(List_price * Quantity * (1-(Discount_Percent/100))) as TotalSales
from Order_Sales
group by Year(Order_Date),Month(Order_Date),datename(month,Order_Date)
order by Year,Month;
"""
df_monthly = pd.read_sql(query_monthly, engine)
plt.plot(df_monthly["MonthName"], df_monthly["TotalSales"], marker='o')
plt.title("Monthly Sales Trend")
plt.xticks(rotation=45)
plt.savefig("visuals/monthly_sales_trend.png")
plt.close()

In [ ]:
query_growth = """
with MonthlySales as (
    select Year(Order_Date) as Year,
    Month(Order_Date) as Month,
    SUM(List_price * Quantity * (1-(Discount_Percent/100))) as MonthlySales
    from Order_Sales group by Year(Order_Date),Month(Order_Date)
)
select a.Year,a.Month,a.MonthlySales as CurrentSales,
       b.MonthlySales as PreviousSales,
       (a.MonthlySales-b.MonthlySales)*100.0/nullif(b.MonthlySales,0) as GrowthPercent
from MonthlySales a
left join MonthlySales b on a.Month=b.Month and a.Year=b.Year+1
order by a.Year,a.Month;
"""
df_growth = pd.read_sql(query_growth, engine)
plt.bar(df_growth["Year"].astype(str)+"-"+df_growth["Month"].astype(str),
        df_growth["GrowthPercent"])
plt.title("Year-over-Year Growth %")
plt.xticks(rotation=90)
plt.savefig("visuals/yoy_growth.png")
plt.close()

In [ ]:
# ----- Quarterly Sales -----
query_quarter = """
select Year(Order_Date) as Year,
Datepart(Quarter , Order_Date) as Quarter,
SUM(List_price * Quantity * (1-(Discount_Percent/100))) as TotalSales
from Order_Sales group by Year(Order_Date),Datepart(Quarter,Order_Date)
order by Year,Quarter;
"""
df_quarter = pd.read_sql(query_quarter, engine)
plt.bar(df_quarter["Year"].astype(str)+"-Q"+df_quarter["Quarter"].astype(str),
        df_quarter["TotalSales"])
plt.title("Quarterly Sales")
plt.savefig("visuals/quarterly_sales.png")
plt.close()